<a href="https://colab.research.google.com/github/raghavmundra/Covid_Data_Visualiztion/blob/main/covid_data_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing required libraries

In [ ]:
import pandas as pd
from bs4 import *
import requests
import json
import folium
import geopy
from geopy.geocoders import Nominatim

##Webscrapping data

In [ ]:
#Fetching covid vaccination data
path1='https://www.mygov.in/sites/default/files/covid/vaccine/vaccine_counts_today.json'
page1 = requests.get(path1)
soup1 = BeautifulSoup(page1.content, 'html.parser')
site_json=json.loads(soup1.text)
site_json['vacc_st_data'][0]
vaccine_data=pd.DataFrame(columns=['state','dose1','dose2','total_doses'])
c=0
for i in site_json['vacc_st_data']:
  vaccine_data.loc[c]=[i['covid_state_name'],i['dose1'],i['dose2'],i['total_doses']]
  c+=1
#Fetching covid cases data
path2='https://www.mygov.in/sites/default/files/covid/covid_state_counts_ver1.json'
page2 = requests.get(path2)
soup2 = BeautifulSoup(page2.content, 'html.parser')
site_json2=json.loads(soup2.text)
active_cases=site_json2['Active']
state_names=site_json2['Name of State / UT']
total_deaths=site_json2['Death']
cured_cases=site_json2['Cured/Discharged/Migrated']
total_cases=site_json2['Total Confirmed cases']
covid_data=pd.DataFrame(columns =['state', 'total_cases','active_cases','discharged_cases','total_deaths'])
covid_data.loc[0]=[state_names['0'],total_cases['0'],active_cases['0'],cured_cases['0'],total_deaths['0']]
for i in range(0,36):
  covid_data.loc[i]=[state_names[str(i)],total_cases[str(i)],active_cases[str(i)],cured_cases[str(i)],total_deaths[str(i)]]

##Data Exploration

In [ ]:
covid_data

,state,total_cases,active_cases,discharged_cases,total_deaths
0,Andaman and Nicobar,7651,5,7517,129
1,Andhra Pradesh,2065716,4837,2046512,14367
2,Arunachal Pradesh,55143,108,54755,280
3,Assam,610150,3791,600367,5992
4,Bihar,726085,45,716379,9661
5,Chandigarh,65340,30,64490,820
6,Chhattisgarh,1005985,300,992110,13575
7,Dadra and Nagar Haveli and Daman and Diu,10682,6,10672,4
8,Delhi,1439788,334,1414363,25091
9,Goa,178063,411,174289,3363


In [ ]:
vaccine_data

,state,dose1,dose2,total_doses
0,Andaman and Nicobar,293904,198731,492635
1,Andhra Pradesh,32814038,20072881,52886919
2,Arunachal Pradesh,771430,531959,1303389
3,Assam,20106832,7902711,28009543
4,Bihar,49637936,18014002,67651938
5,Chandigarh,925246,541908,1467154
6,Chhattisgarh,14777967,7226577,22004544
7,Dadra and Nagar Haveli and Daman and Diu,660648,369465,1030113
8,Delhi,13013583,7364070,20377653
9,Goa,1258903,892242,2151145


In [ ]:
covid_data[['dose1','dose2','total_doses']] = vaccine_data[['dose1','dose2','total_doses']]
covid_data.head()

,state,total_cases,active_cases,discharged_cases,total_deaths,dose1,dose2,total_doses
0,Andaman and Nicobar,7651,5,7517,129,293904,198731,492635
1,Andhra Pradesh,2065716,4837,2046512,14367,32814038,20072881,52886919
2,Arunachal Pradesh,55143,108,54755,280,771430,531959,1303389
3,Assam,610150,3791,600367,5992,20106832,7902711,28009543
4,Bihar,726085,45,716379,9661,49637936,18014002,67651938


##Date Visualization

In [ ]:
india=folium.Map(location = [22,78.9629],zoom_start=4.5)

In [ ]:
geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")
covid_data['gcode'] = covid_data.state.apply(geolocator.geocode)
covid_data['lat'] = [g.latitude for g in covid_data.gcode]
covid_data['long'] = [g.longitude for g in covid_data.gcode]
covid_data.head()


,state,total_cases,active_cases,discharged_cases,total_deaths,dose1,dose2,total_doses,gcode,lat,long
0,Andaman and Nicobar,7651,5,7517,129,293904,198731,492635,"(Andaman, India, (10.0001051, 93.0000194))",10.000105,93.000019
1,Andhra Pradesh,2065716,4837,2046512,14367,32814038,20072881,52886919,"(Andhra Pradesh, India, (15.9240905, 80.1863809))",15.924091,80.186381
2,Arunachal Pradesh,55143,108,54755,280,771430,531959,1303389,"(Arunachal Pradesh, India, (28.0937702, 94.592...",28.093770,94.592133
3,Assam,610150,3791,600367,5992,20106832,7902711,28009543,"(Assam, India, (26.4073841, 93.2551303))",26.407384,93.255130
4,Bihar,726085,45,716379,9661,49637936,18014002,67651938,"(Bihar, India, (25.6440845, 85.906508))",25.644085,85.906508


In [ ]:
for state,lat,long,total_cases,Death,Recov,Active,dose1,dose2,totaldoses in zip(list(covid_data['state']),list(covid_data['lat']),list(covid_data['long']),
                                                         list(covid_data['total_cases']),list(covid_data['total_deaths']),list(covid_data['discharged_cases']),
                                                         list(covid_data['active_cases']),list(covid_data['dose1']),list(covid_data['dose2']),
                                                         list(covid_data['total_doses'])):
    #for creating circle marker
    folium.CircleMarker(location = [lat,long],
                       radius = 5,
                       color='red',
                       fill = True,
                       fill_color="red").add_to(india)
    #for creating marker
    folium.Marker(location = [lat,long],
                  # adding information that need to be displayed on popup
                  popup=folium.Popup(('<strong><b>State  : '+state+'</strong> <br>' +
                    '<strong><b>Total Cases : '+total_cases+'</strong><br>' +
                    '<strong><font color= red>Deaths : </font>'+Death+'</strong><br>' +
                    '<strong><font color=green>Recoveries : </font>'+Recov+'</strong><br>' +
                    '<strong><b>Active Cases : '+Active+'</strong><br>'+
                    '<strong><b><font color=green>Vaccine data : </font></strong> <br>' +
                    '<strong><b>Dose 1 : '+dose1+'</strong><br>' +
                    '<strong><b>Dose 2  : '+dose2+'</strong><br>' +
                    '<strong><b>Total Doses : '+totaldoses+'</strong><br>'),max_width=200)).add_to(india)

In [ ]:
india.save(outfile= "test.html")

In [ ]:
covid_data['Country']='India'

,state,total_cases,active_cases,discharged_cases,total_deaths,dose1,dose2,total_doses,gcode,lat,long,Country
0,Andaman and Nicobar,7651,5,7517,129,293904,198731,492635,"(Andaman, India, (10.0001051, 93.0000194))",10.000105,93.000019,India
1,Andhra Pradesh,2065716,4837,2046512,14367,32814038,20072881,52886919,"(Andhra Pradesh, India, (15.9240905, 80.1863809))",15.924091,80.186381,India
2,Arunachal Pradesh,55143,108,54755,280,771430,531959,1303389,"(Arunachal Pradesh, India, (28.0937702, 94.592...",28.093770,94.592133,India
3,Assam,610150,3791,600367,5992,20106832,7902711,28009543,"(Assam, India, (26.4073841, 93.2551303))",26.407384,93.255130,India
4,Bihar,726085,45,716379,9661,49637936,18014002,67651938,"(Bihar, India, (25.6440845, 85.906508))",25.644085,85.906508,India


##Saving file for tableau :
https://public.tableau.com/app/profile/raghav.mundra/viz/CovidData_16356264427820/Dashboard1

In [ ]:
covid_data.to_csv("./drive/My Drive/Covid_live_data.csv")